In [1]:
%run ./MovieLens.ipynb

In [2]:
# 设置超参数
d = 20
alpha_u = alpha_v = beta_u = beta_v = 0.01
lr = 0.01
epochs = 50

user_num = 943
item_num = 1682

In [3]:
# 初始化参数
def init():
    mu = (y_ui * ratings).sum() / y_ui.sum()
    bu = np.zeros(user_num + 1, float)
    for u in range(1, user_num + 1):
        bu[u] = (0 if y_ui[u].sum() == 0 else (y_ui[u] * (ratings[u] - mu)).sum() / y_ui[u].sum())
    bi = np.zeros(item_num + 1, float)
    for i in range(1, item_num + 1):
        bi[i] = (0 if y_ui[:,i].sum() == 0 else (y_ui[:,i] * (ratings[:,i] - mu)).sum() / y_ui[:,i].sum())
    U = np.random.rand(user_num + 1, d)
    V = np.random.rand(item_num + 1, d)
    U = (U - 0.5) * 0.01
    V = (V - 0.5) * 0.01
    return mu, bu, bi, U, V

# 预测函数
def predict_rule1(u, i, U, V):
    return U[u] @ V[i].T

def predict_rule2(u, i, mu, bu, bi, U, V):
    return mu + bu[u] + bi[i] + U[u] @ V[i].T

In [4]:
def PSVD():
    ratings_svd = ratings.copy().astype('float')
    for u in range(1, user_num + 1):
        for j in range(1, item_num + 1):
            if y_ui[u][j] == 0:
                ratings_svd[u][j] = r_u[u]
        ratings_svd[u] -= r_u[u]
    U, S, V = np.linalg.svd(ratings_svd)
    S_truncated = np.zeros_like(ratings_svd)
    S_truncated[:d, :d] = np.diag(S[:d])
    ratings_svd = U @ S_truncated @ V
    for u in range(1, user_num + 1):
        ratings_svd[u] += r_u[u]
    return ratings_svd

ratings_svd = PSVD()

In [ ]:
def get_SGD_PMF(lr = lr):
    U, V = init()[3], init()[4]
    train_data = get_train_data()
    n = len(train_data)
    for epoch in range(epochs):
        print(epoch)
        # 打乱训练集
        train_data = shuffle_data(train_data)
        for t in range(n):
            # 随机取数据
            u, i, rating = train_data.iloc[t,:]['userId'], train_data.iloc[t,:]['itemId'], train_data.iloc[t,:]['rating']
            # 计算梯度
            delta_Uu = -(ratings[u][i] - U[u] @ V[i].T) * V[i] + alpha_u * U[u]
            delta_Vi = -(ratings[u][i] - U[u] @ V[i].T) * U[u] + alpha_v * V[i]
            # 更新参数
            U[u] -= lr * delta_Uu
            V[i] -= lr * delta_Vi
        # 学习率下降
        lr *= 0.9
    return U, V

U1, V1 = get_SGD_PMF()

In [ ]:
def get_SGD_RSVD(lr = lr):
    mu, bu, bi, U, V = init()
    train_data = get_train_data()
    n = len(train_data)
    for epoch in range(epochs):
        print(epoch)
        # 打乱训练集
        train_data = shuffle_data(train_data)
        for t in range(n):
            # 随机取数据
            u, i, rating = train_data.iloc[t,:]['userId'], train_data.iloc[t,:]['itemId'], train_data.iloc[t,:]['rating']
            # 计算梯度
            e = ratings[u][i] - predict_rule2(u, i, mu, bu, bi, U, V)
            delta_mu = -e
            delta_bu = -e + beta_u * bu[u]
            delta_bi = -e + beta_v * bi[i]
            delta_Uu = (-e) * V[i] + alpha_u * U[u]
            delta_Vi = (-e) * U[u] + alpha_v * V[i]
            # 更新参数
            mu -= lr * delta_mu
            bu[u] -= lr * delta_bu
            bi[i] -= lr * delta_bi
            U[u] -= lr * delta_Uu
            V[i] -= lr * delta_Vi
        # 学习率下降
        lr *= 0.9
    return mu, bu, bi, U, V

mu, bu, bi, U2, V2 = get_SGD_RSVD()

In [7]:
# 预测规则
def PSVD(u, j):
    return ratings_svd[u][j]

def SGD_PMF(u, j):
    return predict_rule1(u, j, U1, V1)

def SGD_RSVD(u, j):
    return predict_rule2(u, j, mu, bu, bi, U2, V2)

In [8]:
# 输出结果
predict(PSVD, SGD_PMF, SGD_RSVD)

RMSE: 1.0169, MAE: 0.8058
RMSE: 0.9686, MAE: 0.7587
RMSE: 0.9564, MAE: 0.7525
